This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
 

In [1]:
%run 0_parameters.ipynb


This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
       
      
2019-03-13
{'w': 28, 'minibatch_size': 100, 'train_batch_size': 100, 'test_batch_size': 20, 'mean': 0.1307, 'std': 0.3081, 'N_pic': 128, 'offset_std': 8, 'offset_max': 35, 'noise': 0.1, 'contrast': 0.7, 'sf_0': 0.2, 'B_sf': 0.3, 'N_theta': 6, 'N_azimuth': 16, 'N_eccentricity': 10, 'N_phase': 2, 'N_X': 128, 'N_Y': 128, 'rho': 1.41, 'bias_deconv': True, 'dim1': 1000, 'dim2': 1000, 'loss_func': BCEWithLogitsLoss(), 'lr': 0.0001, 'do_adam': False, 'epochs': 20, 'momentum': 0.1, 'n_epochs': 10, 'num_processes': 1, 'no_cuda': True, 'log_interval': 100, 'verbose': 1, 'filename': '../data/2019-03-13', 'seed': 2019}


In [2]:
figname = '../paper/fig_result'

In [3]:
from retina import Display
d = Display(args)

In [ ]:
from retina import Retina
r = Retina(args)

# Network

In [ ]:
from where import Where
where = Where(args, d, r)

Loading accuracy... min, max= 0.0145 0.9852


#### Training

In [ ]:
def test(net, optimizer=where.optimizer,
         vsize=args.N_theta*args.N_azimuth*args.N_eccentricity*args.N_phase,
         asize=args.N_azimuth*args.N_eccentricity):
    
    data, label = next(iter(d.loader_test))
    batch_size = label.shape[0]

    retina_data = np.zeros((batch_size, args.N_phase * args.N_theta * args.N_azimuth * args.N_eccentricity))
    fixmap_data = np.zeros((batch_size, args.N_azimuth * args.N_eccentricity))

    for i in range(batch_size):
            data_fullfield, i_offset, j_offset = d.draw(data[i, 0, :, :].numpy())
            retina_data[i, :]  =  r.retina(data_fullfield)
            accuracy_colliculus[i,:], _ = r.accuracy_fullfield(accuracy_map, i_offset, j_offset)
            
    retina_data = Variable(torch.FloatTensor(retina_data))
    accuracy_colliculus = Variable(torch.FloatTensor(accuracy_colliculus))
    
    with torch.no_grad():
        output = net(retina_data)
        acc, _ = accuracy_gain(output, accuracy_colliculus, batch_size)

    return acc


## Anywhere target, with noise

#### Additive noise + whitening

In [ ]:
filename_train = args.filename + '_train.pt'
%ls -l {filename_train}
%rm {filename_train}
#filename_train = '../data/2019-03-11_CNS.npy'
where.train(filename_train)

ls: ../data/2019-03-13_train.pt: No such file or directory
rm: ../data/2019-03-13_train.pt: No such file or directory


Train Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

# Visualisation of some examples

In [ ]:
data, label = next(iter(d.loader_test))
full, retina_data, accuracy_colliculus = where.minibatch(data)

In [ ]:
data.shape, retina_data.shape, accuracy_colliculus.shape

## knowing the target

In [ ]:
accuracy_colliculus.shape, accuracy_colliculus.min(), accuracy_colliculus.max()

In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F


#for idx in range(args.test_batch_size): #range(3, 7):
for idx in range(3, 7):
    fig = plt.figure(figsize = (15, 8))
    
    ax = fig.add_subplot(151)
    ax = d.show(ax, full[idx]['data_fullfield'])
    ax.set_title(f"i={full[idx]['i_offset']}, j={full[idx]['j_offset']}")
    
    ax = fig.add_subplot(152)
    #ax = r.show(ax, r.retina_invert(retina_data[idx, :].numpy()))
    data_retina = r.retina(full[idx]['data_fullfield'])
    ax = r.show(ax, r.retina_invert(data_retina))
    ax.set_title(f"idx={idx}")
              
    ax = fig.add_subplot(153, projection='polar')
    #col = colliculus_inverse @ a_data_n[idx,0,:]
    #vec_t = Variable(torch.FloatTensor(accuracy_colliculus[idx, :].numpy()))
    #vec_t = vec_t.reshape((1, args.N_azimuth * args.N_eccentricity))
    ax.pcolor(r.theta, r.log_r, accuracy_colliculus[idx, :].numpy().reshape((args.N_azimuth, args.N_eccentricity)), cmap=plt.plasma())

                 
    out_t = where.model(retina_data[idx, :])
    out_sig = F.sigmoid(out_t).detach().numpy()
    #out_t = out_t.reshape((1, args.N_azimuth * args.N_eccentricity))
    #acc, _ = accuracy_gain(out_t, vec_t, 1, full_fixmap = full_fixmap_n[idx, :].reshape((1, -1)))
    #view = r.colliculus_inverse @ out_sig.flatten()

    ax = fig.add_subplot(154, projection='polar')
    ax.pcolor(r.theta, r.log_r, out_sig.reshape((args.N_azimuth, args.N_eccentricity)), cmap=plt.plasma())

    ax = fig.add_subplot(155)
    im = where.extract(full[idx]['data_fullfield'], full[idx]['i_offset'], full[idx]['j_offset'])
    ax.imshow(im, cmap=plt.gray())

    proba = where.classify_what(im)
    pred = proba.argmax() # get the index of the max log-probability
    acc = proba[0, pred]
    result = '' if pred==label[idx] else 'FALSE'
    ax.set_title(f"pred={pred} acc={acc:.2f} {result}")
                 

## NOT knowing the target

In [ ]:
# Predict classes using images from the train set
prediction = where.model(retina_data)


In [ ]:
prediction.shape

In [ ]:
torch.optim.Adam?

In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F


#for idx in range(args.test_batch_size): #range(3, 7):
for idx in range(3, 7):
    fig = plt.figure(figsize = (15, 8))
    
    ax = fig.add_subplot(151)
    ax = d.show(ax, full[idx]['data_fullfield'])
    ax.set_title(f"i={full[idx]['i_offset']}, j={full[idx]['j_offset']}")
    
    ax = fig.add_subplot(152)
    #ax = r.show(ax, r.retina_invert(retina_data[idx, :].numpy()))
    data_retina = r.retina(full[idx]['data_fullfield'])
    ax = r.show(ax, r.retina_invert(data_retina))
    ax.set_title(f"idx={idx}")
              
    ax = fig.add_subplot(153, projection='polar')
    #col = colliculus_inverse @ a_data_n[idx,0,:]
    vec_t = Variable(torch.FloatTensor(accuracy_colliculus[idx, :].numpy()))
    vec_t = vec_t.reshape((1, args.N_azimuth * args.N_eccentricity))
    ax.pcolor(r.theta, r.log_r, vec_t.reshape((args.N_azimuth, args.N_eccentricity)), cmap=plt.plasma())
    out_t = where.model(retina_data[idx, :])
    out_sig = F.sigmoid(out_t).detach().numpy()
    out_t = out_t.reshape((1, args.N_azimuth * args.N_eccentricity))
    #acc, _ = accuracy_gain(out_t, vec_t, 1, full_fixmap = full_fixmap_n[idx, :].reshape((1, -1)))
    view = r.colliculus_inverse @ out_sig.flatten()

    ax = fig.add_subplot(154, projection='polar')
    ax.pcolor(r.theta, r.log_r, out_sig.reshape((args.N_azimuth, args.N_eccentricity)), cmap=plt.plasma())

    ax = fig.add_subplot(155)
    im = where.extract(full[idx]['data_fullfield'], full[idx]['i_offset'], full[idx]['j_offset'])
    ax.imshow(im, cmap=plt.gray())

    proba = where.classify_what(im)
    pred = proba.argmax() # get the index of the max log-probability
    acc = proba[0, pred]
    result = '' if pred==label[idx] else 'FALSE'
    ax.set_title(f"pred={pred} acc={acc:.2f} {result}")
                 

In [ ]:
def test_eccentricities(N_eccentricities=9):
    vsize = args.N_theta*args.N_azimuth*args.N_eccentricity*args.N_phase
    asize = args.N_azimuth*args.N_eccentricity
    
    N_pic_mnist = 28
    N_class_mnist = 10

    test_batch_size = 1000
    
    mem_acc_log = []
    mem_acc_data_log = []
    mem_ref_log = [] 
    mem_ref_data_log = []

    ecc_max=.8
    r_pix = []

    eccentricities = range(N_eccentricities)
    for i_eccentricity in eccentricities:

        eccentricity = ecc_max * (1/args.rho)**(args.N_eccentricity - i_eccentricity)

        radius = args.N_pic / 2 * eccentricity
        print('at scale ', i_eccentricity, ' radius=', radius)
        r_pix.append(radius)

        data, label = next(iter(d.loader_test))

        input_n = np.zeros((args.test_batch_size, 1, vsize))
        a_data = np.zeros((args.test_batch_size, asize))
        full_fixmap_n = np.zeros((args.test_batch_size, args.N_pic**2))

        ref_data = np.zeros(test_batch_size)

        for idx in range(test_batch_size):
            theta = np.random.rand() * 2 * np.pi
            i_offset = int(radius * np.cos(theta))
            j_offset = int(radius * np.sin(theta))

            # changed
            data_fullfield = d.place_object(data[idx, 0, :, :].numpy(), i_offset, j_offset)
            input_n[idx, 0, :]  =  r.retina(data_fullfield)

            a_data[idx, :], accuracy_fullfield_map = r.accuracy_fullfield(accuracy_map, i_offset, j_offset)
            full_fixmap_n[idx, :] = accuracy_fullfield_map.flatten()

            # when we do not do a saccade, the reference accurracy data is pre-computed 
            if radius <= N_pic_mnist:
                ref_data[idx] = accuracy_map[N_pic_mnist - 1 + i_offset, N_pic_mnist - 1 + j_offset]
            else:
                ref_data[idx] = 1 / N_class_mnist      


        in_t = Variable(torch.FloatTensor(input_n))
        out_t = net(in_t)
        a_data_t = Variable(torch.FloatTensor(a_data))
        acc, acc_data = accuracy_gain(out_t, a_data_t, test_batch_size, full_fixmap=full_fixmap_n)
        mem_acc_log += [acc]
        mem_acc_data_log += [acc_data]

        mem_ref_log += [np.mean(ref_data)]
    return r_pix, mem_acc_log, mem_ref_log

N_eccentricities = 9
r_pix, mem_acc_log, mem_ref_log = test_eccentricities(N_eccentricities)

In [ ]:
idx = 30
w = 28
in_t = Variable(torch.FloatTensor(input_n[idx,0,:]))
out_t = net(in_t)
out_sig = F.sigmoid(out_t).detach().numpy()

test = out_sig.reshape((args.N_azimuth, args.N_eccentricity))
indices_ij = np.where(test == max(test.flatten()))
azimuth = indices_ij[0][0]
eccentricity = indices_ij[1][0]
full_masque = r.colliculus[azimuth,eccentricity,:]
indice_move = np.where(full_masque == max(full_masque))
i_move = indice_move[0][0] // 128 - 64
j_move = indice_move[0][0] % 128 - 64
print(i_move, j_move)


fig = plt.figure(figsize = (figwidth, figwidth/2.5))#1.618))
ax_A = plt.subplot(1, 4, 1) 
ax_A = r.show(ax_A, r.retina_invert(input_n[idx, 0, :]))

ax_B = plt.subplot(2, 4, 2, projection='polar', autoscale_on=False)
vec_t = Variable(torch.FloatTensor(a_data_n[idx, 0, :]))
vec_t = vec_t.reshape((1, args.N_azimuth * args.N_eccentricity))
ax_B.pcolor(r.theta, r.log_r, vec_t.reshape((args.N_azimuth, args.N_eccentricity)))
ax_B.grid('off')
#ax.set_rgrids('off')
plt.title('True', fontsize = 14)
ax_B.set_yticklabels([])
ax_B.set_xticklabels([])

ax_Bb = plt.subplot(2, 4, 6, projection='polar')
ax_Bb.pcolor(r.theta, r.log_r, out_sig.reshape((args.N_azimuth, args.N_eccentricity)))
ax_Bb.set_title('Predicted', fontsize = 14)
ax_Bb.set_yticklabels([])
ax_Bb.set_xticklabels([])


ax_C = plt.subplot(1, 4, 3)
data_fullfield = d.place_object(data[idx, 0, :, :].numpy(), 0, 0)
input_vector  =  r.retina(data_fullfield)
ax_C = r.show(ax_C, r.retina_invert(input_vector))
ax_A.arrow(64.5, 64.5, j_move, i_move, width=.3, color='r', head_width=4., length_includes_head=True, edgecolor='k')
vec_t = vec_t.reshape((1, args.N_azimuth * args.N_eccentricity))
out_t = out_t.reshape((1, args.N_azimuth * args.N_eccentricity))
acc, _ = accuracy_gain(out_t, vec_t, 1, full_fixmap = full_fixmap_n[idx, :].reshape((1, -1)))

ax_D = plt.subplot(1, 4, 4)
r_ref = r_pix
r_ref = range(9)

ax_D.bar(r_ref, mem_acc_log, alpha = .5, label = 'One saccade')
ax_D.bar(r_ref, mem_ref_log, alpha = .5, label = 'No saccade') #accuracy_map[27,27:55])
ax_D.plot(np.arange(10)-.5, [0.1]*10, ':', c='k', label = 'Baseline')
plt.legend(loc='best')
ax_D.set_title('Class accuracy', fontsize = 14)
ax_D.set_xlabel('Target eccentricity (pixels)', fontsize = 12)
ax_D.set_xticks(range(9))
ax_D.set_xticklabels(['%.1f' % d for  d in r_pix])
ax_D.set_ylim([0,1])

for ax, text in [[ax_A, 'DIS'], [ax_C, 'SAC']]:
    ax.text(4, 15, text, fontsize=24,
          bbox={'facecolor':'white','alpha':1,'edgecolor':'none','pad':1},
          ha='left', va='center') 

offset = -.015
for ax, text, x_offset, y_offset in [[ax_A, 'A', offset, 1.15], [ax_B, 'B', -.25, 1.225], [ax_C, 'C', offset, 1.15], [ax_D, 'D', offset, 1.15]]:
    ax.text(x_offset, y_offset, '(' + text + ')', fontsize=24,
              bbox={'facecolor':'white','alpha':1,'edgecolor':'none','pad':1},
              ha='left', va='center', transform=ax.transAxes) 

# pos : [left, bottom, width, height] =    The new position of the in `.Figure` coordinates.    
plt.tight_layout()
ax_A.set_position([0.025, 0.1, .3, .45])
ax_B.set_position( [0.24, 0.375, .2, 0.2])
ax_Bb.set_position([0.24, 0.1, .2, 0.2])
ax_C.set_position([0.35, .1, .3, .45])
ax_D.set_position([0.65, .1, .3, .45])
fig.savefig(figname + '.pdf', bbox_inches='tight', pad_inches=0.1)

# post-processing

In [ ]:
!convert  -density {dpi_export} {figname}.pdf {figname}.jpg
!convert  -density {dpi_export} {figname}.pdf {figname}.png
#!convert  -density {dpi_export} -resize 5400  -units pixelsperinch -flatten  -compress lzw  -depth 8 {fname}.pdf {fname}.tiff

In [ ]:
from IPython.display import Image
Image('{figname}.png'.format(figname=figname))

In [ ]:
!ls  -l {figname}*

## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts